In [22]:
from typing import Dict, Any, List, Optional
from abc import abstractmethod, ABCMeta
from numbers import Number
import qutip as qt
import numpy as np


# Units and Useful Constants
GHz = 1
MHz = 1e-3
kHz = 1e-6


class Qubit(metaclass=ABCMeta):
    """
    Base class for defining superconducting qubits
    """

    @property
    @abstractmethod
    def expected_params(self) -> List[str]:
        """
        List of expected param strings. Note all frequencies have units GHz.
        E.g. ["Ej", "Ec", "N_max"]
        """

    def __init__(self, params: Dict[str, Any]):
        self.params = params.copy()

        # Set base parameters
        for param in self.expected_params:
            if param not in params:
                raise Exception(f"Please include the required parameter {param}")

        # Define operators
        self.ops: Dict[str, qt.Qobj] = {}
        self.build_ops()
        self._eig_system: Dict[str, List[np.ndarray]] = {}

    # Base Operators - To be overridden
    # ==========================================
    def build_ops(self):
        N = self.params["N_max"]
        self.ops["a_dag"] = qt.create(N)
        self.ops["a"] = qt.destroy(N)

    # System Hamiltonian and Eigenstates
    # ==========================================
    @abstractmethod
    def get_H(self) -> qt.Qobj:
        """
        Method for calculating the static qubit system Hamiltonian. Note: this version takes no args.

        Returns:
            H (qt.Qobj): Qubit Hamiltonian
        """

    @property
    def eig_system(self):
        """
        Method for diagonalizing the system Hamiltonian and storing sorted eigenvalues and eigenvectors.
        """
        N = self.params["N_max"]
        if "vals" not in self._eig_system or "vecs" not in self._eig_system:
            # eig_system = np.linalg.eigh(self.get_H())  #fixed from co pilot suggestion
            eig_system = np.linalg.eigh(self.get_H().full())
            # dim(eigvals_sorted) = N
            eigvals_sorted = np.sort(eig_system[0])
            idxs_sorted_by_eigval = np.argsort(eig_system[0])

            # dim(evs_sorted) = [N, N]
            eigvecs_sorted = eig_system[1][:, idxs_sorted_by_eigval]

            # Truncates eigensystem: keeps only the lowest N eigenvalues and vectors
            self._eig_system["vals"] = np.array(eigvals_sorted)[:N]
            self._eig_system["vecs"] = np.array(eigvecs_sorted)[:, :N]

        return self._eig_system

    @property
    def Ud(self):
        """
        Unitary change of basis matrix consisting of eigenvectors of H₀ as columns.
        """
        return qt.Qobj(self.eig_system["vecs"])

    def op_matrix_in_H_eigenbasis(self, op: qt.Qobj) -> qt.Qobj:
        """
        Method for calculating the matrix of an operator O in the eigenbasis of H₀
        """
        return (self.Ud.dag() * op * self.Ud).tidyup()

    # Analysis
    # ==========================================
    def qubit_report(self):
        """
        Method to print qubit frequency and anharmonicity.
        """
        eigvals = self.eig_system["vals"]

        ω_01 = eigvals[1] - eigvals[0]
        anharm = eigvals[2] + eigvals[0] - 2 * eigvals[1]

        print(f"Qubit frequency ω01 = {ω_01 * GHz} GHz")
        print(f"Qubit anharmonicity α = {anharm / MHz} MHz")
        return ω_01, anharm


class DrivenSystem(metaclass=ABCMeta):
    """
    Base class for a general driven quantum system
    """

    @property
    def expected_params(self) -> List[str]:
        """
        List of expected param strings.
            M_max: Drive "charge" basis truncation, defined -M_max to +M_max
        """
        return ["M_max"]

    def __init__(self, params: Dict[str, Any], qubit: Qubit, drive_coupling: qt.Qobj):
        self.qubit = qubit

        for param in self.expected_params:
            if param not in params:
                raise Exception(f"Please include the required parameter {param}")
        self.params = params.copy()

        # Initialize stored analysis dictionaries
        self.reset_analysis()
        self.ops["drive_coupling"] = drive_coupling

    def reset_analysis(self):
        """
        Initialize stored analysis dictionaries
        """
        self.ops: Dict[str, np.ndarray] = {}
        self.states: Dict[str, np.ndarray] = {}
        self.build_ops_and_states()
        
    
    # Analysis
    # ==========================================
    def get_bare_basis_idxs_overlaps(self, omega: float):
        """
        Method to calculate the indices of the eigenvectors that most closely match |α⟩⊗|m=0⟩,
        at zero drive amplitude Ω = 0. This is done via computing the state overlaps with a 
        fixed basis. 
        """
        H_eff_0 = fq.H_eff(omega, 0)

        eigvals, eigvecs = H_eff_0.eigenstates()

        bare_basis_idxs = []
        for bare_basis_vec in self.states["bare_basis"]:

            overlaps = []
            for vec in eigvecs:
                overlaps.append(np.abs(bare_basis_vec.overlap(vec)))

            max_overlap_idx = np.argmax(overlaps)
            bare_basis_idxs.append(max_overlap_idx)
    
        return np.array(bare_basis_idxs)


In [14]:
class Transmon(Qubit):
    """
    Transmon qubit class. 
    
    Args:
        params (Dict): Dictionary of parameters defining the qubit
    
    On init:
        - Runs build_ops() function. All operators can be saved in dictionary self.ops
        - Generates empty results dictionary self.eig_system 
    """
    
    expected_params = ["Ej", "Ec", "ng", "N_max_charge"]

    def __init__(self, params: Dict[str, Any]):
        # Initialize self.params for storing all system parameters
        self.params = params.copy()
        
        # Set the energy eigenbasis truncation if not specified
        if "N_max" not in self.params:
            self.params["N_max"] = 2 * self.params["N_max_charge"] + 1
        else:
            # Ensure energy eigenbasis dim is not greater than total charge dim
            assert self.params["N_max"] <= 2 * self.params["N_max_charge"] + 1
        super().__init__(self.params)

    
    # Build Transmon Operators
    # ==========================================
    def build_ops(self):
        self.ops["N"] = self.calc_n_op()
        self.ops["cos(φ)"] = self.build_cos_phi_op()
        self.ops["Id_charge"] = qt.qeye(2 * self.params["N_max_charge"] + 1)
        self.ops["Id"] = qt.qeye(self.params["N_max"])

    def calc_n_op(self):
        # We define the charge operator N = ∑ₙ n|n><n| in the charge basis
        N_max_charge = self.params["N_max_charge"]
        return qt.Qobj(np.diag(np.arange(-N_max_charge, N_max_charge + 1)))

    def build_cos_phi_op(self):
        # We define cos(φ) = 1/2 * ∑ₙ|n><n+1| + h.c. in the charge basis
        N_max_charge = self.params["N_max_charge"]
        return 0.5 * qt.Qobj(
            np.eye(2 * N_max_charge + 1, k=1) 
            + np.eye(2 * N_max_charge + 1, k=-1)
        )
    
    @property
    def phi_zpf(self):
        """
        Returns the phase zero-point fluctuation of the transmon.
        """
        Ec = self.params["Ec"]
        Ej = self.params["Ej"]
        return (2*Ec / Ej)**(1/4)
    
    @property
    def n_zpf(self):
        """
        Returns the charge zero-point fluctuation of the transmon.
        """
        Ec = self.params["Ec"]
        Ej = self.params["Ej"]
        return np.sqrt(np.sqrt((Ej / (32*Ec))))
    
    
    # System Hamiltonian Definitions
    # ==========================================
    def get_H(self):
        """
        Method for constructing the transmon Hamiltonian H₀ = 4*Ec*(n - ng)² - Ej*cos(φ) 
        in the charge basis.
        """
        Ej = self.params["Ej"]
        Ec = self.params["Ec"]
        ng = self.params["ng"]

        return (
            4 * Ec * (self.ops["N"] - ng * self.ops["Id_charge"]) ** 2
            - Ej * self.ops["cos(φ)"]
        )

    @property
    def Ud(self):
        """
        Unitary change of basis matrix consisting of eigenvectors of H₀ as columns. 
        """
        return qt.Qobj(self.eig_system['vecs'])
    
    @property
    def H0(self):
        """
        Method to generate the truncated system Hamiltonian H₀ in its energy eigenbasis. 
        Note: this rescales the energies so that the ground state energy is 0.

        TODO: 3 methods to do this: via eigvals, via Ud, via op_matrix function (also Ud)
        """
        eigvals = self.eig_system['vals'] - self.eig_system['vals'][0]
        return qt.Qobj(np.diag(eigvals))

In [27]:

# Fluxonium class 
class Fluxonium(Qubit):
    """
    Fluxonium qubit class.
    
    Args:
        params (Dict): Dictionary of parameters defining the qubit
    
    On init:
        - Runs build_ops() function. All operators can be saved in dictionary self.ops
        - Generates empty results dictionary self._eig_system 
    """
    
    expected_params = ["Ej", "Ec", "El", "ng", "flux", "N_max_harm"]

    def __init__(self, params: Dict[str, Any]):
        # Ensure we have a working params dict and provide sensible defaults
        self.params = params.copy()

        # allow user to pass phi_ext (radians) as alias for flux fraction
        if "flux" not in self.params and "phi_ext" in self.params:
            # interpret phi_ext as radians and convert to reduced flux (Φ_ext/Φ0) fraction
            self.params["flux"] = self.params["phi_ext"] / (2 * np.pi)

        # defaults
        if "ng" not in self.params:
            self.params["ng"] = 0.0

        # harmonic truncation must exist before calling Qubit.__init__ (base class checks expected_params)
        if "N_max_harm" not in self.params:
            # use provided N_max if available, otherwise choose a safe default
            self.params["N_max_harm"] = int(self.params.get("N_max", 30))

        # energy eigenbasis truncation
        if "N_max" not in self.params:
            self.params["N_max"] = self.params["N_max_harm"]
        else:
            assert self.params["N_max"] <= self.params["N_max_harm"]

        # Now call parent constructor which will run build_ops() etc.
        super().__init__(self.params)

    def build_ops(self):
        self.ops["n"] = self.calc_n_op()
        self.ops["phi"] = self.calc_phi_op()
        self.ops["Id_harm"] = qt.qeye(self.params["N_max_harm"])
        self.ops["Id"] = qt.qeye(self.params["N_max"])

    def calc_n_op(self):
        phi_osc = (8 * self.params["Ec"] / self.params["El"]) ** 0.25
        a = qt.destroy(self.params["N_max_harm"])
        return 1j * (a.dag() - a) / (phi_osc * np.sqrt(2))

    def calc_phi_op(self):
        phi_osc = (8 * self.params["Ec"] / self.params["El"]) ** 0.25
        a = qt.destroy(self.params["N_max_harm"])
        return phi_osc * (a + a.dag()) / np.sqrt(2)

    def build_cos_op(self):
        phi = self.ops["phi"]
        Id = self.ops["Id_harm"]
        phi_ext = 2 * np.pi * self.params.get("flux", 0.0)
        arg = phi - phi_ext * Id
        return ((1j * arg).expm() + (-1j * arg).expm()) / 2

    @property
    def phi_zpf(self):
        Ec = self.params["Ec"]
        El = self.params["El"]
        return (8 * Ec / El) ** (1 / 4) / np.sqrt(2)

    @property
    def n_zpf(self):
        Ec = self.params["Ec"]
        El = self.params["El"]
        return 1 / (np.sqrt(2) * (8 * Ec / El) ** (1 / 4))

    def get_H(self):
        Ej = self.params["Ej"]
        Ec = self.params["Ec"]
        El = self.params["El"]
        ng = self.params.get("ng", 0.0)
        return (
            4 * Ec * (self.ops["n"] - ng * self.ops["Id_harm"]) ** 2
            + 0.5 * El * self.ops["phi"] ** 2
            - Ej * self.build_cos_op()
        )

    @property
    def Ud(self):
        # delegate to base-class eig_system to ensure eigenvectors are available
        return qt.Qobj(self.eig_system["vecs"])

    @property
    def H0(self):
        eigvals = np.array(self.eig_system.get("vals", []))
        if eigvals.size == 0:
            # return zero matrix of correct size when eigensystem not available
            return qt.Qobj(np.zeros((self.params["N_max"], self.params["N_max"])))
        eigvals = eigvals - eigvals[0]
        return qt.Qobj(np.diag(eigvals[: self.params["N_max"]]))

In [ ]:
class CoupledQubits:
    def __init__(self, fluxonium_params, transmon_params, Jc):
        self.fluxonium = Fluxonium(fluxonium_params)
        self.transmon = Transmon(transmon_params)
        self.Jc = Jc
        self.build_full_H()
        self.eig_system = self.diagonalize()
    
    def build_full_H(self):
        """Build full coupled Hamiltonian. Robustly handle operator name differences
        between Fluxonium ('n') and Transmon ('N'), and ensure eigenbases are computed
        before forming interaction terms.
        """
        # Ensure eigensystems are computed so Ud/H0 are available
        _ = self.fluxonium.eig_system
        _ = self.transmon.eig_system

        Id_f = qt.qeye(self.fluxonium.params['N_max'])
        Id_t = qt.qeye(self.transmon.params['N_max'])

        H_f = qt.tensor(self.fluxonium.H0, Id_t)
        H_t = qt.tensor(Id_f, self.transmon.H0)

        # Support both 'N' and 'n' keys for number-like operator
        fq_key = 'N' if 'N' in self.fluxonium.ops else ('n' if 'n' in self.fluxonium.ops else None)
        tq_key = 'N' if 'N' in self.transmon.ops else ('n' if 'n' in self.transmon.ops else None)

        if fq_key is None:
            raise KeyError("Fluxonium operator 'N' or 'n' not found in fluxonium.ops")
        if tq_key is None:
            raise KeyError("Transmon operator 'N' or 'n' not found in transmon.ops")

        n_f_op = self.fluxonium.ops[fq_key]
        n_t_op = self.transmon.ops[tq_key]

        # Convert to eigenbasis. Prefer provided helper, fall back to Ud transformation.
        try:
            n_f_eig = self.fluxonium.op_matrix_in_H_eigenbasis(n_f_op)
        except Exception:
            n_f_eig = self.fluxonium.Ud.dag() * n_f_op * self.fluxonium.Ud

        try:
            n_t_eig = self.transmon.op_matrix_in_H_eigenbasis(n_t_op)
        except Exception:
            n_t_eig = self.transmon.Ud.dag() * n_t_op * self.transmon.Ud

        # Build coupling and full Hamiltonian in tensor product space
        H_c = self.Jc * qt.tensor(n_f_eig, n_t_eig)
        self.H_full = H_f + H_t + H_c

        # store dims
        self.dim_f = self.fluxonium.params['N_max']
        self.dim_t = self.transmon.params['N_max']
        self.dim_c = self.dim_f * self.dim_t
        self.H_full.dims = [[self.dim_c], [self.dim_c]]
    
    def diagonalize(self):
        vals, vecs_list = self.H_full.eigenstates(sort='low')
        vecs = np.hstack([v.full() for v in vecs_list])
        return {'vals': vals - vals[0], 'vecs': vecs}  # Ground at 0, vecs as matrix
    
    @property
    def Ud(self):
        return qt.Qobj(self.eig_system['vecs'], dims=[[self.dim_c], [self.dim_c]])
    
    def system_report(self):
        print("Coupled energies (first 5, GHz):", self.eig_system['vals'][:5])


In [29]:
# ...existing code...
import qutip as qt
import numpy as np
import matplotlib.pyplot as plt

# Constants
GHz = 1.0

# Parameters
transmon_params = {'Ec': 0.19 * GHz, 'Ej': 11. * GHz, 'ng': 0.0, 'N_max_charge': 15, 'N_max': 10}
# ensure Fluxonium has the required 'N_max_harm' key (and optionally 'N_max')
fluxonium_params_base = {
    'Ec': 0.91 * GHz,
    'Ej': 3.73 * GHz,
    'El': 0.36 * GHz,
    'N_max_harm': 60,   # harmonic oscillator truncation (required)
    'N_max': 30         # energy-eigenbasis truncation (optional)
}
Jc = 0.05 * GHz

# Sweep phi_ext values
phi_ext_values = np.linspace(0, 2 * np.pi, 100)
eigenvalues = []

for phi_ext in phi_ext_values:
    fluxonium_params = fluxonium_params_base.copy()
    fluxonium_params['phi_ext'] = phi_ext
    coupled = CoupledQubits(fluxonium_params, transmon_params, Jc)
    eigenvalues.append(coupled.eig_system['vals'][:5])  # First 5 eigenvalues

eigenvalues = np.array(eigenvalues)

# Plot
plt.figure(figsize=(8, 6))
for i in range(eigenvalues.shape[1]):
    plt.plot(phi_ext_values, eigenvalues[:, i], label=f"Eigenvalue {i+1}")

plt.xlabel(r"$\phi_{\rm ext}$ (rad)")
plt.ylabel("Eigenvalues (GHz)")
plt.title("Eigenvalues vs $\phi_{\rm ext}$ of Fluxonium")
plt.legend()
plt.grid(True)
plt.show()

<>:40: SyntaxWarning: invalid escape sequence '\p'
<>:40: SyntaxWarning: invalid escape sequence '\p'
/var/folders/r1/k579s3gx3wl29l9z816msv5c0000gn/T/ipykernel_72170/903089412.py:40: SyntaxWarning: invalid escape sequence '\p'
  plt.title("Eigenvalues vs $\phi_{\rm ext}$ of Fluxonium")


KeyError: 'N'

In [ ]:
import qutip as qt
import numpy as np

class Qubit:
    def __init__(self, params):
        self.params = params.copy()
        self.ops = {}
        self.build_ops()
        self.eig_system = self.diagonalize()
    
    def build_ops(self):
        pass  # Subclass
    
    def get_H(self):
        pass  # Subclass
    
    def diagonalize(self):
        H = self.get_H()
        vals, vecs_list = H.eigenstates(sort='low')
        vecs = np.hstack([v.full() for v in vecs_list])
        return {'vals': vals, 'vecs': vecs}
    
    @property
    def Ud(self):
        N_max = self.params['N_max']
        return qt.Qobj(self.eig_system['vecs'][:, :N_max])
    
    @property
    def H0(self):
        eigvals = self.eig_system['vals'][:self.params['N_max']] - self.eig_system['vals'][0]
        return qt.Qobj(np.diag(eigvals))
    
    def qubit_report(self):
        vals = self.eig_system['vals']
        omega01 = vals[1] - vals[0]
        alpha = (vals[2] - vals[1]) - omega01
        print(f"Qubit frequency ω01 = {omega01:.3f} GHz")
        print(f"Qubit anharmonicity α = {alpha * 1000:.3f} MHz")
        return omega01, alpha

class Transmon(Qubit):
    def __init__(self, params):
        if 'ng' not in params:
            params['ng'] = 0.0
        if 'N_max' not in params:
            params['N_max'] = 10
        if 'N_max_charge' not in params:
            params['N_max_charge'] = 15
        super().__init__(params)
    
    def build_ops(self):
        N_max_charge = self.params['N_max_charge']
        self.ops['N'] = qt.Qobj(np.diag(np.arange(-N_max_charge, N_max_charge + 1)))
        self.ops['cos_phi'] = 0.5 * qt.Qobj(np.eye(2 * N_max_charge + 1, k=1) + np.eye(2 * N_max_charge + 1, k=-1))
        self.ops['Id_charge'] = qt.qeye(2 * N_max_charge + 1)
        self.ops['Id'] = qt.qeye(self.params['N_max'])
    
    def get_H(self):
        Ec = self.params['Ec']
        Ej = self.params['Ej']
        ng = self.params['ng']
        return 4 * Ec * (self.ops['N'] - ng * self.ops['Id_charge'])**2 - Ej * self.ops['cos_phi']

class Fluxonium(Qubit):
    def __init__(self, params):
        if 'phi_ext' not in params:
            params['phi_ext'] = np.pi/2  # Sweet spot
        if 'N_max' not in params:
            params['N_max'] = 30  # Larger for fluxonium, adjust if slow
        super().__init__(params)
    
    @property
    def phi_zpf(self):
        return (self.params['Ec'] / self.params['El'])**0.25 / np.sqrt(2)
    
    @property
    def n_zpf(self):
        return (self.params['El'] / self.params['Ec'])**0.25 / np.sqrt(2)
    
    @property
    def omega_lc(self):
        return np.sqrt(8 * self.params['Ec'] * self.params['El'])
    
    def build_ops(self):
        N = self.params['N_max']
        a = qt.destroy(N)
        ad = a.dag()
        self.ops['phi'] = self.phi_zpf * (a + ad)
        self.ops['N'] = 1j * self.n_zpf * (ad - a)  # Hermitian
        self.ops['Id'] = qt.qeye(N)
        self.ops['Id_charge'] = self.ops['Id']  # Compatibility
    
    def get_H(self):
        a = qt.destroy(self.params['N_max'])
        H_lc = self.omega_lc * a.dag() * a
        phi_op = self.ops['phi'] + self.params['phi_ext'] * self.ops['Id']
        # Manual cos
        cos_phi_op = ((1j * phi_op).expm() + (-1j * phi_op).expm()) / 2
        H_j = -self.params['Ej'] * cos_phi_op
        return H_lc + H_j

class CoupledQubits:
    def __init__(self, fluxonium_params, transmon_params, Jc):
        self.fluxonium = Fluxonium(fluxonium_params)
        self.transmon = Transmon(transmon_params)
        self.Jc = Jc
        self.build_full_H()
        self.eig_system = self.diagonalize()
    
    def build_full_H(self):
        """Build full coupled Hamiltonian. Robustly handle operator name differences
        between Fluxonium ('n') and Transmon ('N'), and ensure eigenbases are computed
        before forming interaction terms.
        """
        # Ensure eigensystems are computed so Ud/H0 are available
        _ = self.fluxonium.eig_system
        _ = self.transmon.eig_system

        Id_f = qt.qeye(self.fluxonium.params['N_max'])
        Id_t = qt.qeye(self.transmon.params['N_max'])

        H_f = qt.tensor(self.fluxonium.H0, Id_t)
        H_t = qt.tensor(Id_f, self.transmon.H0)

        # Support both 'N' and 'n' keys for number-like operator
        fq_key = 'N' if 'N' in self.fluxonium.ops else ('n' if 'n' in self.fluxonium.ops else None)
        tq_key = 'N' if 'N' in self.transmon.ops else ('n' if 'n' in self.transmon.ops else None)

        if fq_key is None:
            raise KeyError("Fluxonium operator 'N' or 'n' not found in fluxonium.ops")
        if tq_key is None:
            raise KeyError("Transmon operator 'N' or 'n' not found in transmon.ops")

        n_f_op = self.fluxonium.ops[fq_key]
        n_t_op = self.transmon.ops[tq_key]

        # Convert to eigenbasis. Prefer provided helper, fall back to Ud transformation.
        try:
            n_f_eig = self.fluxonium.op_matrix_in_H_eigenbasis(n_f_op)
        except Exception:
            n_f_eig = self.fluxonium.Ud.dag() * n_f_op * self.fluxonium.Ud

        try:
            n_t_eig = self.transmon.op_matrix_in_H_eigenbasis(n_t_op)
        except Exception:
            n_t_eig = self.transmon.Ud.dag() * n_t_op * self.transmon.Ud

        # Build coupling and full Hamiltonian in tensor product space
        H_c = self.Jc * qt.tensor(n_f_eig, n_t_eig)
        self.H_full = H_f + H_t + H_c

        # store dims
        self.dim_f = self.fluxonium.params['N_max']
        self.dim_t = self.transmon.params['N_max']
        self.dim_c = self.dim_f * self.dim_t
        self.H_full.dims = [[self.dim_c], [self.dim_c]]
    
    def diagonalize(self):
        vals, vecs_list = self.H_full.eigenstates(sort='low')
        vecs = np.hstack([v.full() for v in vecs_list])
        return {'vals': vals - vals[0], 'vecs': vecs}  # Ground at 0, vecs as matrix
    
    @property
    def Ud(self):
        return qt.Qobj(self.eig_system['vecs'], dims=[[self.dim_c], [self.dim_c]])
    
    def system_report(self):
        print("Coupled energies (first 5, GHz):", self.eig_system['vals'][:5])

class DrivenCoupledQubits:
    def __init__(self, coupled, drive_amp, drive_freq, drive_op='transmon_n', N_floquet=5):
        self.coupled = coupled
        self.drive_amp = drive_amp
        self.drive_freq = drive_freq
        self.drive_op = drive_op  # 'transmon_n' or 'fluxonium_n'
        self.N_floquet = N_floquet
        self.build_H_eff()
        self.eig_system = self.diagonalize()
    
    def build_H_eff(self):
        Id_m = qt.qeye(2 * self.N_floquet + 1)
        m_vals = np.arange(-self.N_floquet, self.N_floquet + 1)
        m_op = qt.Qobj(np.diag(m_vals))
        H_c_m = qt.tensor(self.coupled.H_full, Id_m)
        Id_c = qt.qeye(self.coupled.dim_c)
        H_flo = self.drive_freq * qt.tensor(Id_c, m_op)
        # Drive operator in coupled basis
        if self.drive_op == 'transmon_n':
            n_t_eig = self.coupled.transmon.Ud.dag() * self.coupled.transmon.ops['N'] * self.coupled.transmon.Ud
            O_base = qt.tensor(qt.qeye(self.coupled.dim_f), n_t_eig)
            O_base.dims = [[self.coupled.dim_c], [self.coupled.dim_c]]
        elif self.drive_op == 'fluxonium_n':
            n_f_eig = self.coupled.fluxonium.Ud.dag() * self.coupled.fluxonium.ops['N'] * self.coupled.fluxonium.Ud
            O_base = qt.tensor(n_f_eig, qt.qeye(self.coupled.dim_t))
            O_base.dims = [[self.coupled.dim_c], [self.coupled.dim_c]]
        else:
            raise ValueError("Invalid drive_op")
        O_c = self.coupled.Ud.dag() * O_base * self.coupled.Ud
        # Cos term: (exp(iwt) + exp(-iwt))/2 -> off-diagonals
        shift_plus = qt.Qobj(np.eye(2 * self.N_floquet + 1, k=1))
        shift_minus = qt.Qobj(np.eye(2 * self.N_floquet + 1, k=-1))
        H_drive = (self.drive_amp / 2) * qt.tensor(O_c, shift_plus + shift_minus)
        self.H_eff = H_c_m - H_flo + H_drive  # Sign convention for quasienergies
    
    def diagonalize(self):
        vals, vecs_list = self.H_eff.eigenstates(sort='low')
        vecs = np.hstack([v.full() for v in vecs_list])
        return {'vals': vals, 'vecs': vecs}
    
    def floquet_report(self):
        quasi_vals = self.eig_system['vals'] % self.drive_freq
        print("Quasienergies (first 5, mod ω_d, GHz):", quasi_vals[:5])

# Constants
GHz = 1.0
MHz = 1e-3

# Params (adjust as needed)
transmon_params = {'Ec': 0.19 * GHz, 'Ej': 11. * GHz, 'ng': 0.0, 'N_max_charge': 15, 'N_max': 10}
fluxonium_params = {'Ec': 0.91 * GHz, 'Ej': 3.73 * GHz, 'El': 0.36 * GHz, 'phi_ext': np.pi, 'N_max': 30}
Jc = 0.05 * GHz
drive_amp = 0.05 * GHz
drive_freq = 5.0 * GHz  # e.g., near transmon freq

# Reports
print("Transmon:")
_ = Transmon(transmon_params).qubit_report()
print("\nFluxonium:")
_ = Fluxonium(fluxonium_params).qubit_report()

# Coupled
coupled = CoupledQubits(fluxonium_params, transmon_params, Jc)
coupled.system_report()

# Driven
driven = DrivenCoupledQubits(coupled, drive_amp, drive_freq, drive_op='transmon_n', N_floquet=5)
driven.floquet_report()

Transmon:
Qubit frequency ω01 = 3.889 GHz
Qubit anharmonicity α = -215.633 MHz

Fluxonium:
Qubit frequency ω01 = 0.067 GHz
Qubit anharmonicity α = 2053.122 MHz
Coupled energies (first 5, GHz): [0.         0.06666482 2.18570422 2.97886876 3.890163  ]
Quasienergies (first 5, mod ω_d, GHz): [4.99975408 0.0663053  2.18543991 2.97835863 3.88991426]
